In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
!pip install bertopic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 62.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 7.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB

In [5]:
import sys
sys.path.append('/content/drive/MyDrive/nlp_ss24/multilingual-lexical-simplification')
sys.path.append('/content/drive/MyDrive/nlp_ss24/multilingual-lexical-simplification/src')

In [6]:
from llm_lexical_simplifier import LLMLexicalSimplifier
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModelForMaskedLM
from simple_bert_lexical_simplifier import SimpleBertLexicalSimplifier
from benchmark_suite import BenchmarkSuite
from language import Language

# LLM based

In [ ]:
torch.random.manual_seed(0)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    torch_dtype="auto",
    trust_remote_code=True,
    # Flash attention is only supported starting Ampere architecture (ie A100)
    # So we cant use it on free-tier GPUs
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/11.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.2k [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
generation_args = {
    "max_new_tokens": 100,
    "return_full_text": False,
    "do_sample": False,
}

# Define PAD Token = EOS Token
tokenizer.padding_side = "left"
# Define PAD Token = EOS Token
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = model.config.eos_token_id

In [ ]:
de_messages = [
    {"role": "user", "content": "Der Maschinenbau hat in Europa durch die Bildung der EU eine starke Erleichterung erhalten. Die vereinfachte Version des vorigen Satzes ist: Der Maschinenbau hat in Europa durch die Bildung der EU eine starke [MASK] erhalten. Ten results as ordered, AST parsable Python list:"},
    {"role": "assistant", "content": '["Vereinfachung", "Entspannung", "Begünstigung", "Unterstützung", "Rückenwind,  "Förderung", "Rückhalt", "Aufwind", "Antrieb", "Erleichterung"]'},
    {"role": "user", "content": "Die EU-Renaturierungsverordnung galt eigentlich bereits als durchgebracht; das EU-Parlament hatte zugestimmt und die Staaten ihren grundsätzlichen Sanktus signalisiert. Die vereinfachte Version des vorigen Satzes ist: Die EU-Renaturierungsverordnung galt eigentlich bereits als durchgebracht; das EU-Parlament hatte zugestimmt und die Staaten ihren grundsätzlichen [MASK] signalisiert. Ten results as ordered, AST parsable Python list:"},
    {"role": "assistant", "content": '["Zustimmung", "Einverständnis", "Okay", "Segen", "Billigung, Bestätigung, Bejahung, Zuspruch, Akzeptanz, Befürwortung"]'},
]
en_messages = [
    {"role": "user", "content": "The president’s relatives were also said to be critical of the way his closest advisers had prepared him for the debate. The simplified version of the previous sentence is: The president’s relatives were also said to be critical of the way his closest [MASK] had prepared him for the debate. Ten results as ordered, AST parsable Python list:"},
    {"role": "assistant", "content": '["staff", "team", "aides", "consultants", "assistants", "strategists", "supporters", "handlers", "counselors", "staffers"]'},
    {"role": "user", "content": "The intermediate sprint will come 94km or so into the stage. The simplified version of the previous sentence is: The intermediate sprint will come 94km or so into the [MASK]. Ten results as ordered, AST parsable Python list:"},
    {"role": "assistant", "content": '["segment", "part", "section", "phase", "leg", "interval", "course", "route", "portion", "strech"]'},
]
es_messages = [
    {"role": "user", "content": "Los esfuerzos del gobierno para reducir la deuda han sido aplaudidos por la comunidad internacional. La versión simplificada del anterior es: Los esfuerzos del gobierno para reducir la deuda han sido [MASK] por la comunidad internacional. Ten results as ordered, AST parsable Python list:"},
    {"role": "assistant", "content": '["reconocidos", "elogiados", "valorados", "aprobados", "aceptados", "apreciados", "celebrados", "alabados", "felicitados, "notado"]'},
    {"role": "user", "content": "La reunión entre los líderes fue descrita como un éxito por ambas partes. La versión simplificada del anterior es: La reunión entre los líderes fue descrita como un [MASK] por ambas partes. Ten results as ordered, AST parsable Python list:"},
    {"role": "assistant", "content": '["éxito", "logro", "triunfo", "acierto", "avance", "progreso", "éxito rotundo", "consecución", "gran paso", "éxito importante"]'},
]

pt_messages = [
    {"role": "user", "content": "A nova legislação tem como objetivo melhorar a segurança no trabalho. A versão simplificada do anterior é: A nova legislação tem como objetivo melhorar a [MASK] no trabalho. Ten results as ordered, AST parsable Python list:"},
    {"role": "assistant", "content": '["segurança", "proteção", "condição", "segurança ocupacional", "salubridade", "qualidade", "segurança laboral", "condições de trabalho", "prevenção", "resguardo"]'},
    {"role": "user", "content": "O projeto de infraestrutura deve gerar milhares de empregos. A versão simplificada do anterior é: O projeto de infraestrutura deve gerar milhares de [MASK]. Ten results as ordered, AST parsable Python list:"},
    {"role": "assistant", "content": '["empregos", "vagas", "trabalhos", "oportunidades", "postos", "cargos", "colocações", "funções", "contratações", "empregos diretos"]'}
]

llm_ls = LLMLexicalSimplifier(model=model, tokenizer=tokenizer,
                              pattern='{original_sentence}. Die vereinfachte Version des vorigen Satzes ist: {sentence_with_complex_word_masked}. Ten results as ordered, AST parsable Python list:',
                              exemplars=de_messages, mask_token='[MASK]', generation_args=generation_args)

Using cuda for model inference.
Using mask token: "[MASK]".


In [ ]:
suite = BenchmarkSuite(llm_ls, {
        Language.EN: {'pattern': '{original_sentence}. The simplified version of the previous sentence is: '
                                '{sentence_with_complex_word_masked}. Ten results as ordered, AST parsable '
                                'Python list:',
                    'exemplars': en_messages},
        Language.DE: {'pattern': '{original_sentence}. Die vereinfachte Version des vorigen Satzes ist: '
                                 '{sentence_with_complex_word_masked}. Ten results as ordered, AST parsable '
                                 'Python list:',
                    'exemplars': de_messages},
        Language.ES: {'pattern': '{original_sentence}. La versión simplificada del anterior es: '
                                '{sentence_with_complex_word_masked}. Ten results as ordered, AST parsable '
                                'Python list:',
                     'exemplars': es_messages},
        Language.PT: {'pattern': '{original_sentence}. A versão simplificada do anterior é: '
                                '{sentence_with_complex_word_masked}. Ten results as ordered, AST parsable '
                                'Python list:',
                     'exemplars': pt_messages}
        })
suite.run()

Benchmarking model on DE ...
Benchmarking model on GermanEvalDataProvider...


Benchmarking:   0%|          | 2/1040 [00:10<1:33:51,  5.43s/it]

Failed to parse the output from the LLM: The provided string is not a valid list representation:  The Maschinenbau, or machine-building industry, in Europe has benefited significantly from the formation of the European Union (EU). The simplification of the sentence while maintaining the essence of the original statement could be: "Der Maschinenbau hat in Europa durch die Bildung der EU eine starke Unterstützung erhalten." This translates to "The machine-building industry in Europe has received strong support through the formation of the EU."

The ten results as
Returning empty list.


Benchmarking:   0%|          | 4/1040 [00:21<1:38:21,  5.70s/it]

Failed to parse the output from the LLM: The provided string is not a valid list representation:  ["Beruhigung", "Entspannung", "Begünstigung", "Ruhe", "Unterstützung", "Rückhalt", "Aufwind", "Erleichterung", "Befreundung", "Zusicherung", "Zusicherung", "Zusicherung", "Zusicherung", "Zusicherung", "Zusicherung", "Zusicherung", "Zusicherung", "
Returning empty list.


...

Benchmarking:  99%|█████████▉| 1029/1040 [1:13:12<00:57,  5.26s/it]

Failed to parse the output from the LLM: The provided string is not a valid list representation:  ["unvermittelte Gewaltszenen", "unvermittelte Gewaltszenen", "unvermittelte Gewaltszenen", "unvermittelte Gewaltszenen", "unvermittelte Gewaltszenen", "unvermittelte Gewaltszenen", "unvermittelte Gewaltszenen", "unvermittelte Gewaltszenen", "unvermittelte Gewaltszenen", "
Returning empty list.


Benchmarking:  99%|█████████▉| 1030/1040 [1:13:18<00:54,  5.40s/it]

Failed to parse the output from the LLM: The provided string is not a valid list representation:  ["unvermittelte", "unerwartete", "unerwartete", "unerwartete", "unerwartete", "unerwartete", "unerwartete", "unerwartete", "unerwartete", "unerwartete", "unerwartete", "unerwartete", "unerwartete", "unerwartete", "unerwartete", "unerwartete", "unerwart
Returning empty list.


Benchmarking: 100%|██████████| 1040/1040 [1:14:05<00:00,  4.27s/it]


#BERT based

In [7]:
bert_tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")
bert_model = AutoModelForMaskedLM.from_pretrained("google-bert/bert-base-cased")

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at google-bert/bert-base-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [12]:
bert_ls = SimpleBertLexicalSimplifier(bert_model, bert_tokenizer, '[MASK]', '{original_sentence}. The simplified version of the previous sentence is: {sentence_with_complex_word_masked}. Top ten results:', None)

Using mask token: "[MASK]".


In [13]:
suite = BenchmarkSuite(bert_ls,
                       {Language.EN: {'pattern': '{original_sentence}. The simpler version of the previous sentence is: {sentence_with_complex_word_masked}. Top ten results:', 'exemplars': None},
                        Language.DE: {'pattern': '{original_sentence}. Die vereinfachte Version des vorigen Satzes ist: {sentence_with_complex_word_masked}. Die zehn besten Resultate:', 'exemplars': None},
                        Language.ES: {'pattern': '{original_sentence}. La versión simplificada de la frase anterior es: {sentence_with_complex_word_masked}. Los diez primeros resultados:', 'exemplars': None},
                        Language.PT: {'pattern': '{original_sentence}. A versão simplificada da frase anterior é: {sentence_with_complex_word_masked}. Os dez melhores resultados:', 'exemplars': None}})
suite.run()

Benchmarking model on EN ...
Benchmarking model on BenchLSDataProvider...


Benchmarking:  31%|███       | 290/929 [00:04<00:10, 61.86it/s]


KeyboardInterrupt: 